# recurrent neural network (RNN)

In [4]:
import json
import os
from copy import deepcopy
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import torch
import torch.nn as nn

In [5]:
%load_ext autoreload
%autoreload 2
    
torch.manual_seed(1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
device = ("cuda" if torch.cuda.is_available() else "mps"
    if torch.backends.mps.is_available() else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [ ]:
from my_data import MyData

# prepare dataset
infile = 'mature.fa'

md = MyData(infile)
train_data = md.get_data()
print('Number of data:', len(train_data))
print('Observe data:', train_data[0])

In [ ]:
from my_embedding import MyEmbedding

# embedding
me = MyEmbedding(train_data)
train_dataset, valid_dataset = me.split()
me.tokenize()
input_vocab, label_vocab = me.build_vocab()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vocab_size = len(input_vocab)
embed_dim = 24
rnn_hidden_size = 32
fc_hidden_size = 32


model = RNN(vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size) 
model = model.to(device)
model.to

# training model
# update collate_batch
## Step 3-A: define the functions for transformation
# feed a tokens vector representing one sentence 
text_pipeline = lambda x: [input_vocab[token] for token in list(x)]
label_pipeline = lambda x: 1 if x == 'Homo sapiens' else 0
collate_fn = MyCollate(text_pipeline, label_pipeline)

num_epochs = 50
p = MyProcess(model, collate_fn, num_epochs)
trained_model, stat = p.run(train_dataset, valid_dataset)

In [ ]:

print(trained_model.embedding)
trained_model.embedding.weight 

In [ ]:
fig = plt.figure(figsize=(12,4))
x_arr = list(range(num_epochs))
ax = fig.add_subplot(1,2,1)
ax.plot(x_arr, stat.acc_train, label="Train Accuracy")
ax.plot(x_arr, stat.acc_valid, label="Valid Accuracy")
ax.legend(fontsize=12)
# ax.set_ylim(.8, 1)
ax.set_title('Accuracy')
ax.set_xlabel('Epochs')
ax.set_ylabel('Percentage %')

ax = fig.add_subplot(1,2,2)
ax.plot(x_arr, stat.loss_train, label="Train Loss")
ax.plot(x_arr, stat.loss_valid, label="Valid Loss")
ax.legend(fontsize=12)
ax.set_title('Loss')
ax.set_xlabel('Epochs')
ax.set_ylabel('Loss')

plt.show()